In [1]:
%pip install --upgrade datasets
%pip install --upgrade transformers
%pip install --upgrade optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: optuna
    Found existing installation: optuna 4.0.0
    Uninstalling optuna-4.0.0:
      Successfully uninstalled optuna-4.0.0
Note: you may need to restart the kernel to use updated pa

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import Dataset,load_dataset
import ast
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import default_data_collator
import torch
import optuna
import shutil
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlbse-25-dataset/NLBSE_Dataset_Python.csv
Using device: cuda


In [3]:
df = pd.read_csv("/kaggle/input/nlbse-25-dataset/NLBSE_Dataset_Python.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22624 entries, 0 to 22623
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             22624 non-null  object
 1   comment_sentence  22616 non-null  object
 2   labels            22624 non-null  object
 3   types             22624 non-null  object
dtypes: object(4)
memory usage: 707.1+ KB


In [4]:
df.head(10)
# if we are in a kaggle environment we need to use that
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Wandb")
import wandb
# Replace YOUR_API_KEY with your actual API key
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
df.drop_duplicates(subset=['comment_sentence'], keep='first', inplace=True)
df.info()
null_rows = df[df['comment_sentence'].isnull()]

print("Rows with null values in 'comment_sentence':")
print(null_rows)

<class 'pandas.core.frame.DataFrame'>
Index: 16775 entries, 0 to 22623
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             16775 non-null  object
 1   comment_sentence  16774 non-null  object
 2   labels            16775 non-null  object
 3   types             16775 non-null  object
dtypes: object(4)
memory usage: 655.3+ KB
Rows with null values in 'comment_sentence':
               class comment_sentence       labels types
9172  MultiValueDict              NaN  [1 0 0 0 0]    ru


In [6]:
df_cleaned = df.dropna(subset=['comment_sentence'])
print("DataFrame shape after removing nulls:", df_cleaned.shape)
df_cleaned.info()

DataFrame shape after removing nulls: (16774, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 16774 entries, 0 to 22623
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             16774 non-null  object
 1   comment_sentence  16774 non-null  object
 2   labels            16774 non-null  object
 3   types             16774 non-null  object
dtypes: object(4)
memory usage: 655.2+ KB


In [7]:
pattern = r'\*|//'
rows_with_pattern = df_cleaned.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)

# Count rows with patterns
num_rows_with_pattern = rows_with_pattern.sum()
print(f"\nNumber of rows containing patterns: {num_rows_with_pattern}")

# Remove `//` or `*` from all columns
df_cleaned = df_cleaned.replace(pattern, '', regex=True)


Number of rows containing patterns: 17


In [8]:
df = df_cleaned
df['combo'] = df['comment_sentence'] +"  |  "+  df['class']
python_dataset = Dataset.from_pandas(df)
# Split the dataset into train and validation subsets
train_test_split = python_dataset.train_test_split(test_size=0.2, seed=42)

# Extract train and validation datasets
python_train = train_test_split['train']
python_test = train_test_split['test']
python_labels = ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary']

In [9]:
# Use Hugging Face's default data collator
data_collator = default_data_collator
num_labels = len(python_labels)

# Load model
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        "google-bert/bert-base-uncased",
        num_labels=num_labels,
        problem_type="multi_label_classification",
    ).to(device)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["combo"], truncation=True, padding="max_length", max_length=128)
tokenized_train = python_train.map(tokenize_function, batched=True)
tokenized_test = python_test.map(tokenize_function, batched=True)

# Convert labels to tensors
def encode_labels(examples):
    if isinstance(examples['labels'], str):
        examples["labels"]=examples["labels"].replace(" ", ",")
        labels = ast.literal_eval(examples['labels'])
    else:
        labels = examples['labels']
    # Convert labels to tensors
    labels = torch.tensor(labels, dtype=torch.float32)
    return {'labels': labels}
    
tokenized_train = tokenized_train.map(encode_labels)
tokenized_test = tokenized_test.map(encode_labels)

# Format datasets for PyTorch
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# train_dataloader = DataLoader(tokenized_train, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(tokenized_test, batch_size=32, shuffle=False)
def clear_directory(directory):
    """Removes all contents inside a directory."""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove file or symbolic link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove directory
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Define evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    print(f"Logits Shape: {logits.shape}, Labels Shape: {labels.shape}")
    # Apply sigmoid to logits to convert to probabilities
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid function
    preds = (probs > 0.5).astype(int)  # Threshold for multi-label classification
    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    return {"precision": precision, "recall": recall, "f1": f1}
    
def optuna_objective(trial):
    temp_dir = "./temp_results_bert_base"
    if os.path.exists(temp_dir):
        clear_directory(temp_dir)
    else:
        os.makedirs(temp_dir)
    temp_dir = "./temp_logs_bert_base"
    if os.path.exists(temp_dir):
        clear_directory(temp_dir)
    else:
        os.makedirs(temp_dir)
    
    # Define hyperparameter search space
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-4)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)
    batch_size = trial.suggest_categorical("batch_size", [4,8,16, 32])

    # Initialize Trainer with current trial parameters
    training_args = TrainingArguments(
        output_dir="./temp_results_bert_base",  # Temporary directory for checkpoints
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=8,
        weight_decay=weight_decay,
        logging_dir="./temp_logs_bert_base",
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=3,  # Keep only the latest checkpoint
    )

    trainer = Trainer(
        model_init=model_init,  # This makes sure we load the base model at each trial
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
    )

    # Perform training
    trainer.train()
    # Evaluate on the validation set
    eval_results = trainer.evaluate()
    print("Evaluation Results:", eval_results)
    eval_loss = eval_results["eval_loss"]
    # Optuna will minimize this
    return eval_loss

# Run Optuna search
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=12)

# Display best hyperparameters
print("Best Hyperparameters:", study.best_params)
print("Best Evaluation Loss:", study.best_value)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/13419 [00:00<?, ? examples/s]

...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Logits Shape: (3355, 5), Labels Shape: (3355, 5)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


[I 2024-12-01 10:25:18,634] Trial 11 finished with value: 0.0342220813035965 and parameters: {'learning_rate': 4.7056535448679974e-05, 'weight_decay': 0.09752572214731633, 'batch_size': 8}. Best is trial 10 with value: 0.03038754127919674.


Logits Shape: (3355, 5), Labels Shape: (3355, 5)
Evaluation Results: {'eval_loss': 0.0342220813035965, 'eval_precision': 0.9862145023435346, 'eval_recall': 0.9840440165061898, 'eval_f1': 0.985128063894244, 'eval_runtime': 18.1158, 'eval_samples_per_second': 185.198, 'eval_steps_per_second': 11.592, 'epoch': 8.0}
Best Hyperparameters: {'learning_rate': 4.9179071216350686e-05, 'weight_decay': 0.09934669480351677, 'batch_size': 8}
Best Evaluation Loss: 0.03038754127919674
